In [2]:
import undetected_chromedriver as uc
import time
import json
from tqdm import tqdm  # Progress bar
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd 
import os
from bs4 import BeautifulSoup

In [3]:
# Start Chrome
driver = uc.Chrome(headless=False, use_subprocess=True)
# time.sleep(60)  # Allow page to load
#driver.quit()

In [62]:
driver.get("https://www.instagram.com/p/DGcuJ47OgtM/")

In [ ]:
def login():
    driver.get("https://www.instagram.com/accounts/login/")

    driver.find_element(By.CSS_SELECTOR, "body > div.x1n2onr6.xzkaem6 > div.x9f619.x1n2onr6.x1ja2u2z > div > div.x1uvtmcs.x4k7w5x.x1h91t0o.x1beo9mf.xaigb6o.x12ejxvf.x3igimt.xarpa2k.xedcshv.x1lytzrv.x1t2pt76.x7ja8zs.x1n2onr6.x1qrby5j.x1jfb8zj > div > div > div > div > div.x7r02ix.xf1ldfh.x131esax.xdajt7p.xxfnqb6.xb88tzc.xw2csxc.x1odjw0f.x5fp0pe.x5yr21d.x19onx9a > div > button._a9--._ap36._a9_1").click()

    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, "#loginForm > div.x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.xqui205.x1n2onr6.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1 > div:nth-child(5) > button").click()
    time.sleep(1)

    driver.find_element(By.CSS_SELECTOR, "body > div._10.uiLayer._4-hy._3qw > div._59s7._9l2g > div > div > div > div > div:nth-child(3) > div.x1exxf4d.x13fuv20.x178xt8z.x1l90r2v.x1pi30zi.x1swvt13 > div > div:nth-child(1) > div.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x1ypdohk.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1o1ewxj.x3x9cwd.x1e5q0jg.x13rtm0m.x87ps6o.x1lku1pv.x1a2a7pz.x9f619.x3nfvp2.xdt5ytf.xl56j7k.x1n2onr6.xh8yej3 > div > div.x6s0dn4.x78zum5.xl56j7k.x1608yet.xljgi0e.x1e0frkt > div > span > span").click()


    login = driver.find_element(By.XPATH, '//*[@id="email"]')

    login.send_keys("")

    password = driver.find_element(By.XPATH, '//*[@id="pass"]')

    password.send_keys("")
    password.send_keys(Keys.RETURN)

    time.sleep(3)

In [285]:
def get_post_ids(uname):
    """
    Given a username, return a list of post ids after scrolling 5 times
    """
    driver.get(f"https://www.instagram.com/{uname}/")
    time.sleep(2)
    for i in range(5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    posts = [i['href'] for i in soup.find_all("a") if f"{uname}/p/" in i['href'] or f'{uname}/reel/' in i['href']]
    return posts

In [ ]:
def get_profile_details(soup):
    profile_details = {}
    num_posts, num_followers, num_following = [i.getText() for i in soup.find("ul").find_all("li")]
    bio = soup.find("ul").parent.next_sibling.getText(separator=" ")
    profile_details["num_posts"] = num_posts.split(" ")[0]
    profile_details["num_followers"] = num_followers.split(" ")[0]
    profile_details["num_following"] = num_following.split(" ")[0]
    profile_details["bio"] = bio
    profile_details['profile_picture'] = soup.find("img")['src']
    profile_details["verified"] = False
    titles = soup.find_all("title")

    for title in titles:
        if "Verified" in title.getText():
            profile_details["verified"] = True
            break
    
    return profile_details

In [ ]:

def save_json(data, filename):
    """Append a JSON object to a JSON file, creating the file if it doesn't exist."""
    # Check if the file exists
    if os.path.exists(filename):
        # Load existing data
        with open(filename, "r", encoding="utf-8") as file:
            try:
                existing_data = json.load(file)
                if not isinstance(existing_data, list):  # Ensure it's a list
                    existing_data = [existing_data]
            except json.JSONDecodeError:
                existing_data = []  # If file is empty or invalid

        # Append new data
        existing_data.append(data)

        # Save updated JSON
        with open(filename, "w", encoding="utf-8") as file:
            json.dump(existing_data, file, indent=4, ensure_ascii=False)
    
    else:
        # If file doesn't exist, create it and write the data as a list
        with open(filename, "w", encoding="utf-8") as file:
            json.dump([data], file, indent=4, ensure_ascii=False)

    print(f"Data saved to {filename}")


In [119]:
def process_profile(uname):
    driver.get(f"https://www.instagram.com/{uname}/")
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    profile_details = get_profile_details(soup)
    posts = get_post_ids(uname)
    profile_details["posts"] = posts

    save_json(profile_details, "profiles.json")
    return profile_details

In [316]:
from bs4 import BeautifulSoup
import hashlib
import json

def load_comments():
    comments_section = driver.find_element(By.XPATH, '//section/main//div/div[1]/div/div[2]')

    while True:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", comments_section)
        time.sleep(2)  

        try:
            load_more_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Load more comments')]")
            driver.execute_script("arguments[0].click();", load_more_button)  
            print("Clicked 'Load more comments' button")
            time.sleep(2)  
        except NoSuchElementException:
            prev_height = driver.execute_script("return arguments[0].scrollHeight", comments_section)
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", comments_section)
            time.sleep(2)
            new_height = driver.execute_script("return arguments[0].scrollHeight", comments_section)

            if prev_height == new_height:  
                print("Reached the end of comments.")
                break

def extract_comment(comment, hidden=False):
    try:
        username = comment.find("a")['href'].strip().replace("/", "")
        thumbnail_tag = comment.find("img")
        thumbnail = thumbnail_tag["src"] if thumbnail_tag else None  

        comment_text = comment.find("span", class_='_ap3a _aaco _aacu _aacx _aad7 _aade')
        comment_text = comment_text.text.strip() if comment_text else None

        comment_image = comment.find("img")["src"] if comment.find("img") else None

        spans = comment.find_all("span")
        num_likes, english, num_replies = None, True, None
        for span in spans:
            text = span.getText(separator=' ')
            if text.endswith('likes'):
                num_likes = int(text.split(" ")[0])
            if "Translation" in text:
                english = False
            if "replies" in text:
                num_replies = int(text.split(" ")[-1].replace("(", "").replace(")", ""))

        time_tag = comment.find("time")
        commented_time = time_tag.text.strip() if time_tag else None

        comment_id = hashlib.md5(comment_text.encode()).hexdigest() if comment_text else None

        return {
            "username": username,
            "thumbnail": thumbnail,
            "comment": comment_text,
            "comment_image": comment_image,
            "likes": num_likes,
            "replies": num_replies,
            "time": commented_time,
            "english": english,
            "comment_id": comment_id,
            "hidden": hidden
        }
    except Exception as e:
        print(f"Error extracting comment: {e}")
        return None
    
def get_comments(comment_objects, hidden=False):
    extracted_comments = [extract_comment(comment, hidden=hidden) for comment in comment_objects if extract_comment(comment, hidden=hidden)]
    return extracted_comments

def get_all_comments():
    load_comments()
    comments_html = driver.find_element(By.XPATH, "//article//ul").get_attribute("outerHTML")
    soup = BeautifulSoup(comments_html, "html.parser")

    comment_objects = soup.find_all('div', recursive=False)
    extracted_comments = get_comments(comment_objects)

    try:
        hidden_button = driver.find_element(By.XPATH, "//span[contains(., 'View hidden comments')]")
        driver.execute_script("arguments[0].click();", hidden_button)
        time.sleep(2)  
    except NoSuchElementException:
        print("No hidden comments found.")

    load_comments()
    hidden_comments_html = driver.find_element(By.XPATH, "//article//ul").get_attribute("outerHTML")
    soup_hidden = BeautifulSoup(hidden_comments_html, "html.parser")
    hidden_comment_objects = soup_hidden.find_all('div', recursive=False)
    extracted_hidden_comments = get_comments(hidden_comment_objects, hidden=True)

    extracted_comments.extend(extracted_hidden_comments)
    return extracted_comments

def save_comments(postId, comments):
    filename = f"comments/{postId}_comments.json"
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(comments, file, indent=4, ensure_ascii=False)
    print(f"Comments saved to {filename}")  

In [271]:
def download_post(postId):
    import requests
    if "/p/" in postId:
        img_tag = soup.find("article").find("img")
        img_url = img_tag['src']
        alt_text = img_tag.get("alt", "No description available")
         # Extract post ID from URL
        post_id = postId.split("/")[-2]
        save_path = f"downloads/{post_id}"
        os.makedirs(save_path, exist_ok=True)

        # Download the image
        img_data = requests.get(img_url).content
        with open(f"{save_path}/{post_id}.jpg", "wb") as img_file:
            img_file.write(img_data)

        # Save alt text (caption)
        with open(f"{save_path}/{post_id}.txt", "w", encoding="utf-8") as txt_file:
            txt_file.write(alt_text)
        

    else:
        import instaloader

        # Create an Instaloader instance
        loader = instaloader.Instaloader()

        # Reel ID (shortcode from URL)
        reel_id = postId.split("/")[-2]
        # Define the download path
        download_path = f"downloads/{reel_id}"

        # Create the directory if it doesn't exist
        os.makedirs(download_path, exist_ok=True)

        # Change Instaloader's download directory
        loader.dirname_pattern = download_path  # Ensures the reel is saved in its own folder

        # Download the reel
        post = instaloader.Post.from_shortcode(loader.context, reel_id)
        loader.download_post(post, target=download_path)


In [279]:
userlist = [
    'https://www.instagram.com/littlemissflint/?hl=en#',
    'https://www.instagram.com/_morganmcguire_/?hl=en#',
    'https://www.instagram.com/anokhina_elizabeth_2007/?hl=en#',
    'https://www.instagram.com/nianaguerrero/?hl=en#',
    'https://www.instagram.com/deja_clarkk/?hl=en#',
    'https://www.instagram.com/everleighrose/?hl=en#',
    'https://www.instagram.com/rateel.alshehri/?hl=en#',
    'https://www.instagram.com/shakhedc/#', 
    'https://www.instagram.com/immirahbadkid/?hl=en#',
    'https://www.instagram.com/iamzainabfaisal/?hl=en#',
    'https://www.instagram.com/kaitrumpgolfer/?hl=en#',
    'https://www.instagram.com/ananyasharmamusic/?hl=en#',
    'https://www.instagram.com/kennedy_curves_/?hl=en#',
    'https://www.instagram.com/lessiesjournal/?hl=en#',
    'https://www.instagram.com/anna.simone.s/?hl=en#',
    'https://www.instagram.com/mohamed_baidani/?hl=en#',
    'https://www.instagram.com/amirelates/?hl=en#',
    'https://www.instagram.com/ambardriscoll/',
    'https://www.instagram.com/mayandnate/',
    'https://www.instagram.com/luluscorneroftheworld/', 
    'https://www.instagram.com/alwaysalittleextra/', 
    'https://www.instagram.com/bishamberdas/', 
    'https://www.instagram.com/thechubbytwirler/',
    'https://www.instagram.com/posh_heat/',
    'https://www.instagram.com/adityamadiraju/' 
]

users = [user.split("/")[-2] for user in userlist]

In [287]:
login()

In [ ]:
# Step 1: get profile details
for user in tqdm(users):
    process_profile(user)

  4%|▍         | 1/25 [00:15<06:12, 15.52s/it]

Data saved to profiles.json


  8%|▊         | 2/25 [00:30<05:55, 15.47s/it]

Data saved to profiles.json


 12%|█▏        | 3/25 [00:46<05:39, 15.43s/it]

...

Data saved to profiles.json


 96%|█████████▌| 24/25 [06:11<00:15, 15.45s/it]

Data saved to profiles.json


100%|██████████| 25/25 [06:27<00:00, 15.48s/it]

Data saved to profiles.json


In [ ]:
# Step 2: read the profile details
with open("profiles.json", "r", encoding="utf-8") as file:
    profiles = json.load(file)

In [292]:
fnames = os.listdir("downloads")

In [297]:
skipped = []

In [298]:
# Step 3: download the posts
for profile in tqdm(profiles):
    for post in tqdm(profile["posts"]):
        if post.split('/')[-2] not in fnames:
            try:
                driver.get(f"https://www.instagram.com{post}")
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                download_post(post)
            except:
                skipped.append(post)
                print(f"Error downloading post: {post}")
                continue

100%|██████████| 41/41 [00:00<00:00, 167445.44it/s]
JSON Query to graphql/query: 401 Unauthorized - "fail" status, message "Please wait a few minutes before you try again." when accessing https://www.instagram.com/graphql/query [retrying; skip with ^C]
JSON Query to graphql/query: 401 Unauthorized - "fail" status, message "Please wait a few minutes before you try again." when accessing https://www.instagram.com/graphql/query?variables=%7B%22shortcode%22%3A%22CmfRbtOoYPt%22%7D&doc_id=8845758582119845&server_timestamps=true [retrying; skip with ^C]


Error downloading post: /everleighrose/reel/CmfRbtOoYPt/


JSON Query to graphql/query: 401 Unauthorized - "fail" status, message "Please wait a few minutes before you try again." when accessing https://www.instagram.com/graphql/query [retrying; skip with ^C]
JSON Query to graphql/query: 401 Unauthorized - "fail" status, message "Please wait a few minutes before you try again." when accessing https://www.instagram.com/graphql/query?variables=%7B%22shortcode%22%3A%22CkHb1FzjoZO%22%7D&doc_id=8845758582119845&server_timestamps=true [retrying; skip with ^C]


Error downloading post: /everleighrose/reel/CkHb1FzjoZO/


JSON Query to graphql/query: 401 Unauthorized - "fail" status, message "Please wait a few minutes before you try again." when accessing https://www.instagram.com/graphql/query [retrying; skip with ^C]
JSON Query to graphql/query: 401 Unauthorized - "fail" status, message "Please wait a few minutes before you try again." when accessing https://www.instagram.com/graphql/query?variables=%7B%22shortcode%22%3A%22CclMGRnJhUF%22%7D&doc_id=8845758582119845&server_timestamps=true [retrying; skip with ^C]


...

 96%|█████████▌| 24/25 [02:55<00:00,  2.23it/s]

Error downloading post: /posh_heat/p/CwhAUiLrBkP/
Error downloading post: /posh_heat/reel/CwWtlLjI3Yj/
Error downloading post: /posh_heat/p/CwJw0ZCLQoy/
Error downloading post: /posh_heat/reel/Cv9DCMNt-Rp/


Error downloading post: /adityamadiraju/reel/DGHFJReOo5f/
Error downloading post: /adityamadiraju/p/DGE-X38OU8i/
Error downloading post: /adityamadiraju/p/DGBNfUxOq7F/
Error downloading post: /adityamadiraju/reel/DF-eIywJq87/
Error downloading post: /adityamadiraju/reel/DF9QRl8R_NF/
Error downloading post: /adityamadiraju/reel/DF26C1_OiUS/
Error downloading post: /adityamadiraju/reel/DF1c2Ziu3cp/
Error downloading post: /adityamadiraju/reel/DF0RORPOOp8/
Error downloading post: /adityamadiraju/reel/DFyh7PXO8yT/
Error downloading post: /adityamadiraju/reel/DFxqiEWO8GN/
Error downloading post: /adityamadiraju/reel/DFvHb4NuWrd/
Error downloading post: /adityamadiraju/reel/DFs01-3PCng/
Error downloading post: /adityamadiraju/reel/DFsYjBtuqIe/
Error downloading post: /adityamadiraju/p/DFqcxTrv3xU/
Error downloading post: /adityamadiraju/reel/DFp8RCpOtkL/
Error downloading post: /adityamadiraju/reel/DFolfk8O0f6/
Error downloading post: /adityamadiraju/reel/DFn9tFmvjjF/
Error downloading post:

100%|██████████| 25/25 [02:55<00:00,  7.02s/it]

Error downloading post: /adityamadiraju/reel/DE-PUjoOIbd/
Error downloading post: /adityamadiraju/reel/DE8mB8huVeA/
Error downloading post: /adityamadiraju/reel/DE5N8k8u1XN/
Error downloading post: /adityamadiraju/reel/DE48DyguUhf/
Error downloading post: /adityamadiraju/reel/DE3jOYOOOXY/


In [300]:
login()

In [307]:
driver.refresh()

In [311]:
load_comments()

Reached the end of comments.


In [340]:
def scroll_comments():
    while True:
        try:
            # Find the comments section
            comment_section = driver.find_element(By.XPATH, '//*[@id="mount_0_0_2n"]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div[2]/div[1]/article/div/div[2]/div/div[2]/div[1]/ul/div[3]/div')  # Adjust selector if needed

            # Scroll down using JavaScript
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", comment_section)
            time.sleep(2)

            # Click "Load more comments" if available
            load_more_button = driver.find_elements(By.XPATH, "//span[contains(text(),'View more comments')]")
            if load_more_button:
                load_more_button[0].click()
                time.sleep(2)
            else:
                break  # Exit if no more comments to load

        except Exception as e:
            print("Error:", e)
            break

In [341]:
scroll_comments()

In [339]:
comment_section = driver.find_element(By.CSS_SELECTOR, '#mount_0_0_z8 > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div > div.x78zum5.xdt5ytf.x1t2pt76.x1n2onr6.x1ja2u2z.x10cihs4 > div.x9f619.xvbhtw8.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1qughib > div.x1gryazu.xh8yej3.x10o80wk.x14k21rp.x17snn68.x6osk4m.x1porb0y.x8vgawa > section > main > div._aa6b._ad9f._aa6d > div._aa6e > article > div > div.x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1n2onr6.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1 > div > div.x78zum5.xdt5ytf.x1q2y9iw.x1n2onr6.xh8yej3.x9f619.x1iyjqo2.x18l3tf1.x26u7qi.xy80clv.xexx8yu.x4uap5.x18d9i69.xkhd6sd > div.x78zum5.xdt5ytf.x1iyjqo2.xs83m0k.x2lwn1j.x1odjw0f.x1n2onr6.x9ek82g.x6ikm8r.xdj266r.x11i5rnm.x4ii5y1.x1mh8g0r.xexx8yu.x1pi30zi.x18d9i69.x1swvt13 > ul > div:nth-child(2) > div > div > div > div > div')  # Adjust selector if needed

# Scroll down using JavaScript
driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", comment_section)
time.sleep(2)

In [329]:
driver.get("https://www.instagram.com/littlemissflint/p/DBrDSAexZpS/")

In [331]:
load_comments()

Error during scrolling: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]"}
  (Session info: chrome=134.0.6998.45); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   undetected_chromedriver             0x0000000104b20922 undetected_chromedriver + 5843234
1   undetected_chromedriver             0x0000000104b185ea undetected_chromedriver + 5809642
2   undetected_chromedriver             0x00000001045f4dfe undetected_chromedriver + 421374
3   undetected_chromedriver             0x0000000104648b69 undetected_chromedriver + 764777
4   undetected_chromedriver             0x0000000104648f51 undetected_chromedriver + 765777
5   undetected_chromedriver             0x0000000104698664 undetected_chromedriver + 1091172
6   undetected_chromedriver     

In [325]:
comments_section = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]')


In [326]:
driver.execute_script("return arguments[0].scrollHeight", comments_section)

2004

In [327]:
driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", comments_section)

In [301]:
# Step 4: get comments 

for profile in tqdm(profiles):
    for post in tqdm(profile["posts"]):
        driver.get(f"https://www.instagram.com{post}")
        time.sleep(2)
        comments = get_all_comments()
        save_comments(post.split("/")[-2], comments)
        time.sleep(2)

  0%|          | 0/25 [00:03<?, ?it/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#mount_0_0_Fk > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div > div.x78zum5.xdt5ytf.x1t2pt76.x1n2onr6.x1ja2u2z.x10cihs4 > div.x9f619.xvbhtw8.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1qughib > div.x1gryazu.xh8yej3.x10o80wk.x14k21rp.x17snn68.x6osk4m.x1porb0y.x8vgawa > section > main > div._aa6b._ad9f._aa6d > div._aa6e > article > div > div.x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1n2onr6.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1 > div > div.x78zum5.xdt5ytf.x1q2y9iw.x1n2onr6.xh8yej3.x9f619.x1iyjqo2.x18l3tf1.x26u7qi.xy80clv.xexx8yu.x4uap5.x18d9i69.xkhd6sd > div.x78zum5.xdt5ytf.x1iyjqo2.xs83m0k.x2lwn1j.x1odjw0f.x1n2onr6.x9ek82g.x6ikm8r.xdj266r.x11i5rnm.x4ii5y1.x1mh8g0r.xexx8yu.x1pi30zi.x18d9i69.x1swvt13 > ul > div:nth-child(3)"}
  (Session inf

In [290]:
profiles[0]

{'num_posts': '1,993',
 'num_followers': '191K',
 'num_following': '1,835',
 'bio': 'Mari Copeny Threads littlemissflint Public figure Mari aka Little Miss Flint.  Future President. Cheerleader. Role Model. Influencer.  #BGCAmbassador   Link icon linktr.ee/Littlemissflint',
 'profile_picture': 'https://scontent-fra5-1.cdninstagram.com/v/t51.2885-19/440322848_1482077482681992_3814007692134227375_n.jpg?stp=dst-jpg_s150x150_tt6&_nc_ht=scontent-fra5-1.cdninstagram.com&_nc_cat=100&_nc_oc=Q6cZ2AHXC6_z9LrZ9QVHV2GjZExgqDjTeU5CBHnVr0eY1MxUu-e4onq0IbmHyBY9bKbdQYc&_nc_ohc=wktu0GZaxT4Q7kNvgG_f1Jz&_nc_gid=536b3cd908c946b4b0a10a47bbd2004b&edm=AP4sbd4BAAAA&ccb=7-5&oh=00_AYFrrX3QbNTmlLZfxgHpGhfE1ahcZ22U0QirQsmSr7ZMrA&oe=67D3D6EB&_nc_sid=7a9f4b',
 'posts': ['/littlemissflint/p/DB10Al_xh50/',
  '/littlemissflint/p/DB0HsS_RBC4/',
  '/littlemissflint/p/DBrDSAexZpS/',
  '/littlemissflint/p/DBmIATdxVwC/',
  '/littlemissflint/reel/DBmFLaXxt6W/',
  '/littlemissflint/p/DBh9Bj1xy9m/',
  '/littlemissflint/p/DBXu

In [80]:
# Get the comments
def get_comments(post_id):
    driver.get(f"https://www.instagram.com{post_id}")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    comments = [i.text for i in soup.find_all("span", class_="gElp9")]
    return comments

In [81]:
get_comments(posts[0])

[]

In [121]:
driver.get(f"https://www.instagram.com{posts[0]}")

In [ ]:
comments_section = driver.find_element(By.CSS_SELECTOR, '#mount_0_0_PW > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div > div.x78zum5.xdt5ytf.x1t2pt76.x1n2onr6.x1ja2u2z.x10cihs4 > div.x9f619.xvbhtw8.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1qughib > div.x1gryazu.xh8yej3.x10o80wk.x14k21rp.x17snn68.x6osk4m.x1porb0y.x8vgawa > section > main > div._aa6b._ad9f._aa6d > div._aa6e > article > div > div.x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1n2onr6.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1 > div > div.x78zum5.xdt5ytf.x1q2y9iw.x1n2onr6.xh8yej3.x9f619.x1iyjqo2.x18l3tf1.x26u7qi.xy80clv.xexx8yu.x4uap5.x18d9i69.xkhd6sd > div.x78zum5.xdt5ytf.x1iyjqo2.xs83m0k.x2lwn1j.x1odjw0f.x1n2onr6.x9ek82g.x6ikm8r.xdj266r.x11i5rnm.x4ii5y1.x1mh8g0r.xexx8yu.x1pi30zi.x18d9i69.x1swvt13 > ul')

for _ in range(2):  # Adjust the range for more scrolling
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", comments_section)
    time.sleep(2)  # Wait for more comments to load


Clicked 'Load more comments' button
Clicked 'Load more comments' button
Clicked 'Load more comments' button
Clicked 'Load more comments' button
Clicked 'Load more comments' button
Clicked 'Load more comments' button
Clicked 'Load more comments' button
Clicked 'Load more comments' button
Clicked 'Load more comments' button
Clicked 'Load more comments' button
Clicked 'Load more comments' button
Clicked 'Load more comments' button
Clicked 'Load more comments' button
Reached the end of comments.


In [251]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [255]:
soup.find("article").find("img")['src']

'https://scontent-fra3-1.cdninstagram.com/v/t51.2885-15/482993788_18493353967059374_7565227470484902809_n.jpg?stp=dst-jpg_e35_tt6&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi4xMDgweDEzNTAuc2RyLmY3NTc2MS5kZWZhdWx0X2ltYWdlIn0&_nc_ht=scontent-fra3-1.cdninstagram.com&_nc_cat=1&_nc_oc=Q6cZ2AHdjYAaoY-jGUeGRiCENjw-VLHYOSbwsEG5dbfnCXHeMG0BBV-M62L_M6l7bZTZQfk&_nc_ohc=3pssZ-2tiYcQ7kNvgHrFNix&_nc_gid=e88e48ff98da47218f39ea8ba9f2cb51&edm=AP4sbd4BAAAA&ccb=7-5&ig_cache_key=MzU4MjAzOTQ2MzA0NDU2OTY2Nw%3D%3D.3-ccb7-5&oh=00_AYDfNY5Vp5lbekOamx_t2dnNgzbyET7336V14RaTswNuVw&oe=67CF25C4&_nc_sid=7a9f4b'

In [ ]:
a

data 

    -- comments.csv (append)
    -- users.csv
    -- posts.csv 
    -- media/
        -- postXYZ.jpg
        -- postXYY.mp4
    

In [ ]:
import os

# Create an Instaloader instance
loader = instaloader.Instaloader()

# Reel ID (shortcode from URL)
reel_id = "DGyMFCXKt9X"

# Define the download path
download_path = f"downloads/{reel_id}"

# Create the directory if it doesn't exist
os.makedirs(download_path, exist_ok=True)

# Change Instaloader's download directory
loader.dirname_pattern = download_path  # Ensures the reel is saved in its own folder

# Download the reel
post = instaloader.Post.from_shortcode(loader.context, reel_id)
loader.download_post(post, target=download_path)

print(f"Download complete! Files saved in: {download_path}")

downloads/DGyMFCXKt9X/2025-03-04_16-31-13_UTC.jpg [President Donald Trump has or…] downloads/DGyMFCXKt9X/2025-03-04_16-31-13_UTC.mp4 json 
Download complete! Files saved in: downloads/DGyMFCXKt9X


In [ ]:
#

In [ ]:
def extract_comment_details(soup):
    # Extract user thumbnail
    user_thumbnail = soup.find("img", class_="xpdipgo")["src"] if soup.find("img", class_="xpdipgo") else None

    # Extract username (usually inside an <a> tag)
    username_tag = soup.select_one("a._a6hd")
    username = username_tag.text.strip() if username_tag else None

    # Extract comment text (should come after username)
    comment_tag = soup.find("span", class_="x1lliihq x1plvlek xryxfnj x1n2onr6 x1ji0vk5 x18bv5gf x193iq5w xeuugli x1fj9vlw x13faqbe x1vvkbs x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x1i0vuye xvs91rp xo1l8bm x5n08af x10wh9bi x1wdrske x8viiok x18hxmgj")  #("span.x1lliihq.x1plvlek")
    comment = comment_tag.text.strip() if comment_tag else None

    # Extract number of likes
    likes_tag = soup.select_one("span.x6ikm8r")
    likes = likes_tag.text.strip() if likes_tag else "0 likes"  # Default to 0 if not found

    return {
        "thumbnail": user_thumbnail,
        "username": username,
        "comment": comment,
        "likes": likes
    }


In [8]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [17]:
coomments = soup.find('div', class_='x78zum5 xdt5ytf x1iyjqo2').find_all("div", class_="x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh x1uhb9sk x1plvlek xryxfnj x1c4vz4f x2lah0s xdt5ytf xqjyukv x1qjc9v5 x1oa3qoh x1nhvcw1")

In [10]:
comments = soup.find_all("div", class_="x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh x1uhb9sk x1plvlek xryxfnj x1c4vz4f x2lah0s xdt5ytf xqjyukv x1qjc9v5 x1oa3qoh x1nhvcw1")

In [11]:
len(comments)

1871

In [40]:
comments[1].find_all("span")[-8].get_text()

'The leader of Hanoi has spoken'

In [46]:
comments[5].find_all("span")[-1].get_text()

'❤️❤️🙌Jane Fonda; you r awesome, amazing and always tell the truth 👏🔥'

In [25]:
extract_comment_details(comments[1])

{'thumbnail': 'https://scontent-fra5-2.cdninstagram.com/v/t51.2885-19/482128978_1027739396083147_6639733810311271523_n.jpg?stp=dst-jpg_s150x150_tt6&_nc_ht=scontent-fra5-2.cdninstagram.com&_nc_cat=107&_nc_oc=Q6cZ2AHWA8SnnvKxCbUdsFOyWFoDOvpv-s9FW0O-F1EPkrgQRtWYiaMOH36Rhjpt55KZEpI&_nc_ohc=lWidVN4VodsQ7kNvgEIHPrs&_nc_gid=b30040398c2c49d984bdaecbfc2a0809&edm=AKp6CbIBAAAA&ccb=7-5&oh=00_AYBz_HScIIEIYTnpr8U51_zkeIbghsR1FP5HPL_2CPzzNw&oe=67CCF1E2&_nc_sid=d62176',
 'username': '',
 'comment': 'timothyferris63',
 'likes': '58 likes'}

In [ ]:
# User (poster) data
    ## Username
    ## User thumbnail
    ## Number of followers, followings
    ## Number of posts
    ## Bio
    ## checkmarked
    ## User ID

# Post data

    ## download the video/image
    ## Caption 
    ## post ID 
    ## User ID
    ## Number of likes
    ## Number of comments
    ## date 


# Comments (ordered)
    ## username
    ## user thumbnail
    ## comment
    ## number of likes
    ## number of replies
    ## date (1 w, 2w)
    ## comment id (if there is, otherwise create unique id)
    ## post ID 



# TODO: run langdetect 
